In [ ]:
import numpy as np
import pandas as pd
import json

In [ ]:
train_set_org = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\additional_resources\\2019-master\data\\train.csv',
                       sep=',',
                      nrows=100,
                      index_col=0)
train_set_org[:3]

In [ ]:
train = train_set_org[train_set_org['action_type'] == 'clickout item']

In [ ]:
train = train.drop(['action_type', 'timestamp', 'step'], axis=1)

In [ ]:
train[:5]

In [ ]:
# dodanie properties o hotelu z id w kolumnie reference

In [ ]:
item_metadata_org = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\item_metadata_separated_properties.csv',
                       sep=',')
# item_metadata_org.set_index('item_id')
item_metadata_org[:5]

In [ ]:
item_metadata_org = item_metadata_org.rename(columns={"item_id": "reference"})
item_metadata_org = item_metadata_org.set_index('reference')
item_metadata_org[:5]

In [ ]:
new_train = train.copy()

In [ ]:
new_train['reference'] = new_train['reference'].astype(int)
new_train = new_train.join(item_metadata_org, on='reference')
new_train[:5]

In [ ]:
new_train.shape

In [ ]:
# usunięcie wierszy z hotelami, o których nie mamy informacji w pliku item_metadata
new_train = new_train.loc[(new_train['Guest House'] ==  1.0) | (new_train['Guest House'] ==  0.0)]

In [ ]:
new_train.shape

In [ ]:
# new_train.iloc[1:,9:].isna().sum()
# new_train.loc[(new_train['Guest House'] !=  1.0) & (new_train['Guest House'] !=  0.0)]

In [ ]:
new_train_without_properties = new_train.iloc[:,:9]
new_train_only_properties = new_train.iloc[:,9:].astype(int)
new_train = pd.concat([new_train_without_properties, new_train_only_properties], axis=1, sort=False)
new_train[:5]

In [ ]:
##################### podział impressions i prices
new_train_as_dict_of_dicts = new_train.to_dict(orient='index')
for item_id, value in new_train_as_dict_of_dicts.items():
    list_of_impressions = value['impressions'].split('|')
    list_of_prices = value['prices'].split('|')

    counter = 0
    for impression in list_of_impressions:
        new_train_as_dict_of_dicts[item_id]['rank_' + str(counter)] = impression
    for price in list_of_prices:
        new_train_as_dict_of_dicts[item_id]['price_' + str(counter)] = price
        counter = counter + 1

new_df = pd.DataFrame.from_dict(new_train_as_dict_of_dicts, orient='index')
new_df = new_df.drop(['impressions', 'prices'], axis=1)
new_df[:5]

In [ ]:
new_df.iloc[:,-25:]

In [ ]:
def get_popularity(df):
    """Get number of clicks that each item received in the df."""

    mask = df["action_type"] == "clickout item"
    df_clicks = df[mask]
    df_item_clicks = (
        df_clicks
        .groupby("reference")
        .size()
        .reset_index(name="n_clicks")
        .transform(lambda x: x.astype(int))
    )

    return df_item_clicks


In [ ]:
df_pop = get_popularity(merged)
merged_test = df_pop.reset_index().merge(
    merged[['reference', 'properties_sum']],
    left_on='reference',
    right_on='reference',
    how='left')
merged_test[:5]

In [ ]:
import math
def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out

In [ ]:
col_expl = 'impressions'
df = train_set_org.copy()
df.loc[:, col_expl] = df[col_expl].apply(string_to_array)  # zamienia 1|2|3 na [1,2,3]

df_out = pd.DataFrame(
    {col: np.repeat(df[col].values,
                    df[col_expl].str.len())
     for col in df.columns.drop(col_expl)}
)

df_out.loc[:, col_expl] = np.concatenate(df[col_expl].values)
df_out.loc[:, col_expl] = df_out[col_expl].apply(int)
df_out[:50]

In [ ]:
item_metadata_org.iloc[:5, 1:]